# 예제로 배우는 Pytorch


PyTorch의 핵심에는 2가지 주요한 특징이 있습니다:
- NumPy와 유사하지만 GPU 상에서 실행 가능한 N차원 Tensor
- 신경망을 구성하고 학습하는 과정에서의 자동 미분

완전히 연결된 ReLU 신경망을 예제로 사용할 것입니다. 

이 신경망은 하나의 은닉 계층(Hidden Layer)을 갖고 있으며, 신경망의 출력과 정답 사이의 유클리드 거리 (Euclidean Distance)를 최소화하는 식으로 경사하강법(Gradient Descent)을 사용하여 무작위의 데이터를 맞추도록 학습할 것입니다.

# Tensor

## Numpy


PyTorch를 소개하기 전에, 먼저 NumPy를 사용하여 신경망을 구성해보겠습니다.

- NumPy는 N차원 배열 객체(Object)를 제공하며, 이러한 배열을 조작하기 위한 다양한 함수(function)들을 제공합니다. 
- NumPy는 과학적 분야의 연산을 위한 포괄적인 프레임워크 (Framework)입니다; 
- 이는 연산 그래프(Computational Graph)나 딥러닝, 변화도(Gradient)에 대해서는 알지 못합니다. 

하지만 NumPy 연산을 사용하여 순전파 단계와 역전파 단계를 직접 구현함으로써, 2-계층을 갖는 신경망이 무작위의 데이터를 맞추도록 할 수 있습니다:

In [ ]:
import numpy as np

# N은 배치 크기이며, D_in은 입력의 차원
# H는 은닉 계층의 차원이며, D_out은 출력 차원
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성
x = np.random.randn(N, D_in)  # 64x1000
y = np.random.randn(N, D_out) # 64x10

# 무작위로 가중치를 초기화
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

In [ ]:
learning_rate = 1e-6
for t in range(500):
    # 순전파 단계:예측값 y를 계산
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # 손실(Loss)을 계산하고 출력
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # 손실에 따른 w1, w2의 변화도를 계산하고 역전파
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h<0] = 0
    grad_w1 = x.T.dot(grad_h)

    # 가중치를 갱신
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

## Pytorch:Tensor

NumPy는 훌륭한 프레임워크지만, GPU를 사용하여 수치 연산을 가속화할 수는 없습니다. 

현대의 심층 신경망에서 GPU는 종종 50배 또는 그 이상 의 속도 향상을 제공하기 때문에, 안타깝게도 NumPy는 현대의 딥러닝에는 충분치 않습니다.

이번에는 PyTorch의 기본적인 개념인 Tensor 에 대해서 소개하겠습니다. 

> PyTorch Tensor
- 기본적으로 NumPy 배열과 동일합니다
- Tensor는 N차원 배열이며, PyTorch는 Tensor 연산을 위한 다양한 함수들을 제공합니다. 
- NumPy 배열과 같이, PyTorch Tensor는 딥러닝이나 연산 그래프, 변화도는 알지 못하며, 과학적 분야의 연산을 위한 포괄적인 도구입니다.
- 그러나 NumPy와는 달리, PyTorch Tensor는 GPU를 활용하여 수치 연산을 가속화할 수 있습니다. 

GPU에서 PyTorch Tensor를 실행하기 위해서는 단지 새로운 자료형으로 변환(Cast)해주기만 하면 됩니다.

여기에서는 PyTorch Tensor를 사용하여 2-계층의 신경망이 무작위 데이터를 맞추도록 할 것입니다. 

위의 NumPy 예제에서와 같이 신경망의 순전파 단계와 역전파 단계는 직접 구현하겠습니다.

In [ ]:
import torch

dtype = torch.FloatTensor
# dtype = torch.cuda.FloatTensor # GPU에서 실행하기위한 코드

# N은 배치 크기이며, D_in은 입력의 차원입니다
# H는 은닉 계층의 차원이며, D_out은 출력 차원입니다
N, D_in, H, D_out = 64, 1000, 100, 10

# 무작위의 입력과 출력 데이터를 생성합니다.
x = torch.randn(N, D_in).type(dtype)
y = torch.randn(N, D_out).type(dtype)

# 무작위로 가중치를 초기화합니다.
w1 = torch.randn(D_in, H).type(dtype)
w2 = torch.randn(H, D_out).type(dtype)

learning_rate = 1e-6
for t in range(500):
    # 순전파 단계 : 예측값 y를 계산
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # 손실(Loss)을 계산하고 출력
    loss = (y_pred - y).pow(2).sum()
    print(t, loss)

    # 손실에 따른 w1, w2의 변화도를 계산하고 역전파
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h<0] = 0
    grad_w1 = x.t().mm(grad_h)

    # 경사하강법(Gradient Descent)를 사용하여 가중치 갱신
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

# Autograd

## PyTorch: Variables과 autograd



위의 예제에서 우리는 신경망의 순전파 단계와 역전파 단계를 수동으로 구현하였습니다. 

작은 2-계층 신경망에서 역전파 단계를 직접 구현하는 것은 큰 일이 아니지만, 대규모의 복잡한 신경망에서는 매우 아슬아슬한 일일 것입니다.

다행히도, 자동 미분 을 사용하여 신경망에서 역전파 단계의 연산을 자동화할 수 있습니다. 
PyTorch의 autograd 패키지는 이 기능을 정확히 제공합니다. 

Autograd를 사용할 때, 신경망의 순전파 단계는 연산 그래프 를 정의합니다; 

그래프의 노드(Node)는 Tensor이며, 엣지(Edge)는 입력 Tensor로부터 출력 Tensor를 만들어내는 함수입니다. 

이 그래프를 통해 역전파를 하게 되면 변화도를 쉽게 계산할 수 있습니다.
이는 복잡해보이지만 실제로 사용하는 것은 간단합니다. 

PyTorch Tensor를 Variable 객체로 감싸게 되면, 이 Variable이 연산 그래프에서 노드로 표현(represent)됩니다. 
- x 가 Variable일 때, x.data 는 그 값을 갖는 Tensor이며 x.grad 는 어떤 스칼라 값에 대해 x 에 대한 변화도를 갖는 또 다른 Variable 입니다.


PyTorch Variable은 PyTorch Tensor와 동일한 API를 제공합니다: - Tensor에서 할 수 있는 (거의) 모든 연산은 Variable에서도 할 수 있습니다; 
- 차이점은 연산 그래프를 정의할 때 Variable을 사용하면, 자동으로 변화도를 계산할 수 있다는 것입니다.

여기에서는 PyTorch Variable과 autograd를 사용하여 2-계층 신경망을 구현합니다; 

이제 더 이상 신경망의 역전파 단계를 직접 구현할 필요가 없습니다:


In [ ]:
import torch
from torch.autograd import Variable

dtype = torch.FloatTensor
dtype = torch.cuda.FloatTensor # GPU에서 실행할 때

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉 계층의 차원이며, D_out은 출력 차원입니다:
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성하고, Variable로 감쌈
# requires_grade=False로 설정하여 역전파 중에 이 Variable들에 대한 변화도를 계산할 필요가 없음을 나타냄
x = Variable(torch.randn(N, D_in).type(dtype), requires_grad=False)
y = Variable(torch.randn(N, D_out).type(dtype), requires_grad=False)

# 가중치를 저장하기 위해 무작위 값을 갖는 Tensor를 생성하고, Variable로 감쌈
# require_grade=True로 설정하여 역전파 중에 이 Variable들에 대한 변화도를 계산할 필요가 있음을 나타냄
w1 = Variable(torch.randn(D_in, H).type(dtype), requires_grad=True)
w2 = Variable(torch.randn(H, D_out).type(dtype), requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 순전파 단계 : Variable 연산을 사용하여 y값 에측
    # 이는 Tensor를 사용한 순전파 단계와 완전히 동일하지만
    # 역전파 단계를 별도로 구현하지 않기 위해 중간 값들(Intermediate Value)에 대한 참조(Reference)를 갖고있을 필요가 없음
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    

    # Variable 연산을 사용하여 손실을 계산하고 출력
    # Loss는 (1,) 모양을 갖는 Variable이며, Loss.data는 (1,) 모양의 Tensor
    # Loss.data[0]은 손실(loss)의 스칼라 값
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.data)

    # autograde를 사용하여 역전파 단계를 계산
    # 이는 requires_grad=True를 갖는 모든 Variable에 대한 손실의 변화도를 계산
    # 이후 w1.grad와 w2.grad는 w1, w2 각각에 대한 손실의 변화도를 갖는 Variable이 됨
    loss.backward()

    # 경사하강법(Gradient Descent)을 사용하여 가중치를 갱신
    # w1.data, w2.data는 Tensor이며, w1.grad와 w2.grad는 Variable이고, 
    # w1.grad.data, w2.grad.data는 Tensor이다
    w1.data -= learning_rate * w1.grad.data
    w2.data -= learning_rate * w2.grad.data

    # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듬
    w1.grad.data.zero_()
    w2.grad.data.zero_()

## PyTorch: 새 autograd 함수 정의

Under the hood, autograd의 기본(primitive) 연산자는 실제로 Tensor를 조작하는 2개의 함수입니다. 
- forward 함수는 입력 Tensor로부터 출력 Tensor를 계산합니다. 
- backward 함수는 출력 Tensor의 변화도를 받고 입력 Tensor의 변화도를 계산합니다.

PyTorch에서 torch.autograd.Function 의 서브클래스(subclass)를 정의하고 forward 와 backward 함수를 구현함으로써 쉽게 사용자 정의 autograd 연산자를 정의할 수 있습니다. 

그 후, 인스턴스(instance)를 생성하고 함수처럼 호출하여 입력 데이터를 포함하는 Variable을 전달하는 식으로 새로운 autograd 연산자를 쉽게 사용할 수 있습니다.

이 예제에서는 ReLU 비선형성(nonlinearity)을 수행하기 위한 사용자 정의 autograd 함수를 정의하고, 2-계층 신경망에 이를 적용해보도록 하겠습니다:

In [ ]:
import torch
from torch.autograd import Variable

class MyReLU(torch.autograd.Function):
    @staticmethod   # staticmethod 아래 정의된 함수를 클래스에 대한 객체 작업을 거치지 않고 클래스 외부에서 사용할 수 있게해줌
                    # staticmethod : 파이썬에서 제공하는 함수
    def forward(ctx, input):
        # 순전파 단계에서는 입력을 갖는 Tensor를 받아 출력을 갖는 Tensor 반환
        # ctx는 역전파 연산을 위한 정보를 저장하기 위해 사용하는 Context Object
        # ctx.save_for_backward method를 사용하여 역전파 단계에서 사용할 어떠한 객체도 저장(cache)해 둘 수 있음
        ctx.save_for_backward(input)
        return input.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        # 역전파 단계에서는 출력에 대한 손실의 변화도를 갖는 Tensor를 받고,
        # 입력에 대한 손실의 변화도를 계산
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input<0] = 0
        return grad_input

dtype = torch.FloatTensor

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉 계층의 차원이며, D_out은 출력 차원입니다:
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성하고, Variable로
# 감쌉니다.
x = Variable(torch.randn(N, D_in).type(dtype), requires_grad=False)
y = Variable(torch.randn(N, D_out).type(dtype), requires_grad=False)

# 가중치를 저장하기 위해 무작위 값을 갖는 Tensor를 생성하고, Variable로
# 감쌉니다.
w1 = Variable(torch.randn(D_in, H).type(dtype), requires_grad=True)
w2 = Variable(torch.randn(H, D_out).type(dtype), requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 사용자 정의 함수를 적용하기 위해 Function.apply method를 사용
    # 이를 'relu'라고 이름(alias) 붙임
    relu = MyReLU.apply

    # 순전파 단계: Variable 연산을 사용하여 y 값 예측
    # 사용자 정의 autograd 연산을 사용하여 ReLU를 계산
    y_pred = relu(x.mm(w1)).mm(w2)

    # 손실(Loss)를 계산하고 출력
    loss = (y_pred - y).pow(2).sum()
    print(t,loss.data)

    # autograde를 사용하여 역전파 단계 계산
    loss.backward()

    # 경사하강법(Gradient Descent)를 사용하여 가중치를 갱신
    w1.data -= learning_rate * w1.grad.data
    w2.data -= learning_rate * w2.grad.data

    # 가중치 갱신 후에는 수동으로 변화도를 0으로 만듭니다.
    w1.grad.data.zero_()
    w1.grad.data.zero_()

# nn 모듈

## PyTorch:nn

연산 그래프와 autograd는 복잡한 연산자를 정의하고 도함수(derivative)를 자동으로 계산하는데 매우 강력한 패러다임(paradigm)입니다; 

하지만 규모가 큰 신경망에서는 autograd 그 자체만으로는 너무 낮은 수준(low-level)일 수 있습니다.

신경망을 구성할 때 종종 연산을 여러 계층 으로 배열하는 것으로 생각하게 되는데, 이 중 일부는 학습 도중 최적화가 될 학습 가능한 매개변수 를 갖고 있습니다.

Tensorflow에서 Keras, TensorFlow-Slim, 나 TFLearn 같은 패키지는 원초적(raw)인 연산 그래프보다 더 높은 수준의 추상화(higher-level abstraction)를 제공하여 신경망을 구축하는데 있어 유용합니다.

PyTorch에서는 nn 패키지가 동일한 목적으로 제공됩니다. 
- nn 패키지는 대략 신경망 계층들과 동일한 모듈 의 집합을 정의합니다. 
- 모듈은 입력 Variable을 받고 출력 Variable을 계산하는 한편, 학습 가능한 매개변수를 포함하는 Variable과 같은 내부 상태(internal state)를 갖습니다. 
- 또한, nn 패키지는 신경망을 학습시킬 때 주로 사용하는 유용한 손실 함수들도 정의합니다.

이번 예제에서는 nn 패키지를 사용하여 2-계층 신경망을 구성해보겠습니다:

In [ ]:
import torch
from torch.autograd import Variable

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉 계층의 차원이며, D_out은 출력 차원입니다:
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성하고, Variable로 감쌉니다.
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# nn 패키지를 사용하여 모델을 순차적인 계층(Sequence of layers)으로 정의
# nn.Sequential은 다른 모듈들을 포함하는 모듈로, 그 모듈들을 순차적으로 적용하여 출력을 생성
    # 각각의 선형(Linear) 모듈은 선형 함수를 사용하여 입력으로부터 출력을 계산하고, 
    # 가중치와 편향(Bias)을 저장하기 위해 내부적인 Variable을 갖음
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# 또한, nn 패키지에는 널리 사용하는 손실 함수들에 대한 정의도 포함하고 있음
# 여기에서는 평균 제곱 오차(MSE)를 손실함수로 사용
loss_fn = torch.nn.MSELoss(size_average=False)

learning_rate = 1e-4
for t in range(500):
    # 순전파 단계 : 모델에 x를 전달하여 예상하는 y 값을 계산
    # 모듈 객체는 __call__ 연산자를 Override해서 함수처럼 호출할 수 있게 함
    # 그렇게 함으로써 입력 데이터의 Variable을 모듈에 전달하고 출력 데이터의 Variable을 생성
    y_pred = model(x)

    # 손실을 계산하고 출력.
    # 예측한 y값과 정답 y를 갖는 Variable들을 전달하고, 손실 함수는 손실(Loss)을 갖는 Variable을 반환
    loss = loss_fn(y_pred, y)
    print(t, loss.data)

    # 역전파 단계를 실행하기 전에 변화도를 0으로 만듬
    model.zero_grad()

    # 역전파 단계 : 모델의 학습 가능한 모든 매개변수에 대해서 손실의 변화도를 게산
    # 내부적으로 각 모듈의 매개변수는 requires_grad=True일 때 Variable 내에 저장되므로,
    # 이 호출은 모든 모델의 모든 학습 가능한 매개변수의 변화도를 계산
    loss.backward()

    # 경사하강법(Gradient Descent)를 사용하여 가중치를 갱신
    # 각 매개변수는 Variable이므로 이전에 했던 것과 같이 데이터와 변화도에 접근할 수 있음
    for param in model.parameters():
        param.data -= learning_rate * param.grad.data

## PyTorch:optim

지금까지는 학습 가능한 매개변수를 갖는 Variable의 .data 멤버를 직접 조작하여 모델의 가중치를 갱신하였습니다. 

이는 확률적 경사 하강법과 같은 간단한 최적화 알고리즘에서는 크게 부담이 되지는 않지만, 실제로 신경망을 학습할 때는 주로 AdaGrad, RMSProp, Adam 등과 같은 좀 더 정교한 Optimizer를 사용합니다.

PyTorch의 optim 패키지는 최적화 알고리즘의 아이디어를 추상화하고 일반적으로 사용하는 최적화 알고리즘의 구현체(implementation)를 제공합니다.

이 예제에서는 앞에서와 같이 nn 패키지를 사용하여 모델을 정의하지만, optim 패키지가 제공하는 Adam 알고리즘을 이용하여 모델을 최적화하겠습니다:

In [ ]:
import torch
from torch.autograd import Variable

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉 계층의 차원이며, D_out은 출력 차원입니다:
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성하고, Variable로 감쌉니다.
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# nn 패키지를 사용하여 모델과 손실 함수를 정의합니다.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(size_average=False)

# optim 패키지를 사용하여 모델의 가중치를 갱신할 Optimizer를 정의합니다.여기서는 Adam을 사용하겠습니다; 
# optim 패키지는 다른 다양한 최적화 알고리즘을 포함하고 있습니다. 
# Adam 생성자의 첫번째 인자는 갱신해야 하는 Variable을 Optimizer에 알려줍니다.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상하는 y 값을 계산합니다.
    y_pred = model(x)

    # 손실을 계산하고 출력합니다.
    loss = loss_fn(y_pred, y)
    print(t, loss.data)

    # 역전파 단계 전에, Optimizer 객체를 사용하여 (모델의 학습 가능한 가중치인) 갱신할 Variable들에 대한 모든 변화도를 0으로 만듭니다. 
    # 이는 기본적으로, .backward()를 호출할 때마다 변화도가 버퍼(Buffer)에 (덮어쓰지 않고) 누적되기 때문
    # 더 자세한 내용은 torch.autograd.backward에 대한 문서를 참조
    optimizer.zero_grad()

    # 역전파 단계: 모델의 매개변수에 대한 손실의 변화도를 계산합니다.
    loss.backward()

    # Optimizer의 step 함수를 호출하면 매개변수가 갱신됩니다.
    optimizer.step()

## PyTorch: 사용자 정의  nn 모듈

가끔은 기존 모듈의 순차적 구성보다 더 복잡한 모델을 구성해야 할 때가 있습니다; 

이럴 때는 nn.Module 의 서브클래스로 새 모듈을 정의하고, 입력 Variable을 받아 다른 모듈 또는 Variable의 autograd 연산을 사용하여 출력 Variable을 생성하는 forward 를 정의합니다.

이 예제에서는 2-계층 신경망을 사용자 정의 Module의 서브클래스로 구현해보겠습니다:

In [ ]:
import torch
from torch.autograd import Variable

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        # 생성자에서 2개의 nn.Linear 모듈을 생성(Instantiate)하고, 멤버 변수로 지정
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        # 순전파 함수에서는 입력 데이터의 Variable을 받아서 출력 데이터의 Variable을 반환
        # Variable 상의 임의의 연산자뿐만 아니라 생성자에서 정의한 모듈 사용 가능
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉 계층의 차원이며, D_out은 출력 차원입니다:
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성하고, Variable로
# 감쌉니다.
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# 앞에서 정의한 클래스를 생성(Instantiating)해서 모델 구성
model = TwoLayerNet(D_in, H, D_out)

# 손실함수와 Optimizer를 만듦
# SGD 생성자에서 model.parameters()를 호출하면 모델의 멤버인 2개의 nnlinear 모듈의 학습 가능한 매개변수들이 포함됨
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # 순전파 단계
    y_pred = model(x)

    # 손실을 계산하고 출력
    loss = criterion(y_pred, y)
    print(t, loss.data)

    # 변화도를 0으로 만들고, 역전파 단계를 수행하고, 가중치를 갱신
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

## PyTorch: 제어 흐름(Control Flow) + 가중치 공유(Weight Sharing)

동적 그래프와 가중치 공유의 예로, 매우 이상한 모델을 구현해보겠습니다: 
- 각각의 순전파 단계에서 많은 은닉 계층을 갖는 완전히 연결(Fully-connected)된 ReLU 신경망이 무작위로 1 ~ 4 사이의 숫자를 선택하고, 
- 동일한 가중치를 여러 번 재사용하여 가장 안쪽(Innermost)에 있는 은닉 계층들을 계산합니다.

이 모델에서는 반복문을 구현하기 위해 일반적인 Python 제어 흐름을 사용하고, 순전파 단계를 정의할 때 단지 동일한 모듈을 여러번 재사용함으로써 내부(innermost) 계층들 간의 가중치 공유를 구현할 수 있습니다.

이 모델은 간단히 Module을 상속받는 서브클래스로 구현해보겠습니다:

In [ ]:
import random
import torch
from torch.autograd import Variable

class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        # 생성자에서는 순전파 단계에서 사용할 3개의 nn.Linear 개체(Instance)를 생성
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H,H)
        self.output_linear = torch.nn.Linear(H, D_out)
    
    def forward(self, x):
        """
        모델의 순전파 단계에서, 무작위로 0, 1, 2 또는 3 중에 하나를 선택하고
        은닉 계층 표현(representation)을 계산하기 위해 여러번 사용한 middle_linear
        모듈을 재사용합니다.

        각 순전파 단계에서 동적 연산 그래프를 구성하기 때문에, 모델의 순전파 단계를
        정의할 때 반복문이나 조건문과 같이 일반적인 Python 제어 흐름 연산자를 사용할
        수 있습니다.

        여기에서 연산 그래프를 정의할 때 동일한 모듈을 여러번 재사용하는 것이
        완벽하게 안전하다는 것을 알 수 있습니다. 이것이 각 모듈을 한 번만 사용하는
        Lua Torch보다 크게 개선된 부분입니다.
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred

# N은 배치 크기이며, D_in은 입력의 차원입니다;
# H는 은닉 계층의 차원이며, D_out은 출력 차원입니다:
N, D_in, H, D_out = 64, 1000, 100, 10

# 입력과 출력을 저장하기 위해 무작위 값을 갖는 Tensor를 생성하고, Variable로
# 감쌉니다.
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# 앞에서 정의한 클래스를 생성(Instantiating)해서 모델을 구성합니다.
model = DynamicNet(D_in, H, D_out)

# 손실함수와 Optimizer를 만듭니다. 이 이상한 모델을 순수한 확률적 경사 하강법
# (Stochastic Gradient Decent)으로 학습하는 것은 어려우므로, momentum을 사용합니다.
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    # 순전파 단계: 모델에 x를 전달하여 예상하는 y 값을 계산합니다.
    y_pred = model(x)

    # 손실을 계산하고 출력합니다.
    loss = criterion(y_pred, y)
    print(t, loss.data)

    # 변화도를 0으로 만들고, 역전파 단계를 수행하고, 가중치를 갱신합니다.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()